In [1]:
''' This script is developed to study the evolution of the NPDFS 
    OF MOLECULAR CLOUDS for various initial conditons.
    
    
    Date Created : 26April 2019
    Author  : Sayantan
    
    Model : TPB16V2 
    
'''

## Importing the requires modules
%matplotlib inline
import numpy as np
# print(np.__version__)
import os.path
import matplotlib.pyplot as plt
import time as t
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
# plt.rc('text',usetex=True)
import os.path
import glob
import TP_module as mf
import pymc 
from pylab import*
from scipy.optimize import curve_fit
from scipy.stats import chisquare
# plt.style.use('seaborn-darkgrid')
import multiprocessing as mp
print('Running on PyMC3 v{}'.format(pymc.__version__))

TP_module is imported
Running on PyMC3 v2.3.6


In [2]:
## Declaring empty list for the four fit varaibles

x0_mean_evol =[]
x0_std_evol =[]

sigma_mean_evol= []
sigma_std_evol= []

alpha_mean_evol= [] 
alpha_std_evol= [] 

TP_min_mean_evol = []
TP_min_std_evol = []
TIME = []

## addded to track the mean and std for the lognormal fit

LN_x0_mean_evol = []
LN_x0_std_evol = []

LN_sigma_mean_evol =[]
LN_sigma_std_evol = []
TIME_LN =[]


In [3]:
Model = "TP_B16_v20"

current_directory = os.getcwd() 
print("the current directory in", current_directory)

save_path1 = './time_evol_plots_'+ Model
save_path = './fit_params_'+ Model
param_folder= current_directory + '/time_evol_plots_'+ Model
image_folder= current_directory + '/fit_params_'+ Model

# os.path.isdir(path)

## Making the directory for the putput files
try:  
#     os.makedirs(param_folder)
    os.makedirs(image_folder)
except OSError:  
    print ("Creation of the directory %s failed/ not needed as it alread exit")
else:  
    print ("Successfully created the directory %s")

the current directory in /Users/sayantan/Documents/Work/Projects/TRANSITION_PDF_17Dec18/TP_25April_19/TP_V2_21_May19
Creation of the directory %s failed/ not needed as it alread exit


In [4]:
## This read all the column density date from the folder and sort them in time
list_of_files = glob.glob('../evolution_data/'+ Model +'/*.dat')
list = np.sort(list_of_files)
# print(list)
outputfilename1 = os.path.join(save_path,'MCMC_' + Model[3:]+".txt")
outputfilename2 = os.path.join(save_path,'LN' +  Model[3:]+".txt")
outputfilename3 = os.path.join(save_path, Model[3:]+'params'+".jpg")
outputfilename4 =os.path.join(save_path, Model[3:]+'Transition'+".jpg")
outputfilename5 =os.path.join(save_path, Model[3:]+'alpha'+".jpg")

In [5]:
## Various bin sizes: This can be later modified by using other astropy modules

bin = [50,100,150,200,250,295] 
b = bin[5]

In [6]:
def do_plot(number):
    
    '''
    This part of the code is taken from Stackoverflow
    The plot function:
    Input: The length of the data file 
    Output: The images in the specified output folder 
    
    '''

    fileName = list[number]
#     print(filename)
    fig = plt.figure(number)
    print(fileName)
    # real data and lognormal data
    xdata1, ydata1, ydata_not_normalized,norm_const = mf.fittingdata(fileName,cutoff= None,binsize=b,upperlimit=2.0)
    x = xdata1 ## the bincenters after binning in log10
    f = ydata1 ## normalized ydata without the log10 division
   
#     ## to determine the time
    time_step = int(fileName.replace('../evolution_data/'+ Model +'/colden_ev', '').replace('.dat', ''))
    time_in_my = str((np.float(time_step)-10)*2.5*10**(-2))
    print(time_step)
    
#   ## To set the intial guess parameters:
    ### We use the scipy curve fit :
    
    
    ### Model Selection 
    
    '''Important note: For fitting the model we fit the normalised data. The data is not divided by log10 but
                       it is multiplied in the plotting function to avaoid errors.
                       
                       The chisqaure is calculated on the unnormalized data. 
                       The model values are multiplied with the normalization value
    
    '''
    
    ### Model 1 lognormal + power-law
    
    popt_lg, pcov_lg = curve_fit(mf.function_lognormal, xdata1, ydata1,bounds=((0,0),(1,1)))
    error_std_lg = np.sqrt(np.diag(pcov_lg))
    
    ## For saving the mean and the standard deviation
    
    
#     print(popt_lg)
    ## Model 2 lognormal + power-law
    
    popt, pcov = curve_fit(mf.function1, xdata1, ydata1,bounds=((0,0,-7,.5),(1,1,-2,.7)))
    print(popt)
    print(type(popt))
    error_std = np.sqrt(np.diag(pcov))
    
     ##for fitting the fucntions to the data
    
    
    
    ####### 
    
    ## This section let us decide the xlimit fot plotting, such that there
    ## is no negetive frequency
    
    xfit_test = np.linspace(np.min(x), np.max(x), len(ydata1)) ## defining an xarray for the model
    xmax_lg,xmin_lg = mf.fitting_datafor_chisquare(xfit_test,*popt_lg,norm_const)
    
    #####
    
    xfit_lg = np.linspace(xmin_lg, xmax_lg, len(ydata1))
    yfit_lg = mf.lognormal_plot(xfit_lg, *popt_lg,norm_const)
    
    
    xmax_lgpw,xmin_lgpw = mf.fitting_datafor_chisquare_lgpw(xfit_test,*popt,norm_const)
    xfit_lgpw = np.linspace(xmin_lgpw, xmax_lgpw, len(ydata1))
    yfit_lgpw = mf.function_plot(xfit_lgpw, *popt,norm_const)
    
    
    
    chisq_lg = chisquare(ydata_not_normalized, yfit_lg)
    
    chisq_lgpw = chisquare(ydata_not_normalized, yfit_lgpw )
    
    
    
    print("chisq lognormal+powerlaw (LC method)=", chisq_lgpw)
    print("chisq lognormal(LC method)=",chisq_lg )
    

    #     if chisq_lgpw[0] <= 0.8*chisq_lg[0]:

## additional costraint added for bad fits
    if (chisq_lgpw[0] <= 0.8*chisq_lg[0] or chisq_lg[0] >20 or chisq_lgpw[0] > 20) and popt[2]>-5 and popt[3]< np.max(x): 
        print("fit lognormal + Powerlaw")
        
        #Set initial parameters

        p0_center = popt[0]   ## mean 
        p0_width = popt[1]    ## sigma
        p0_slope = popt[2]    ## alpha
        p0_TP = popt[3]     ## Transition point
        
        ### if the curvy fit routines are not senstive to the changes
        ### then the intial conditions are imposed by hand
        
        if popt[3] >=np.max(xdata1) or popt[2]>0:
            p0_TP = x[np.where(f == max(f))][0]+np.std(x)
            p0_slope = -3.
        
        p0 = [p0_center, p0_width, p0_slope, p0_TP]
        print("Initial params",p0)


        ## Implementing the MCMC routine: 

        MDL = pymc.MCMC(mf.model(x, f, p0))
        MDL.sample(20000, 5000, 1)
#         pymc.Matplot.plot(MDL)
        
        
        
        ### The fit parameters :

        x0_mean, x0_std = MDL.stats()['x0']['mean'], MDL.stats()['x0']['standard deviation']
        sigma_mean, sigma_std = MDL.stats()['sigma']['mean'], MDL.stats()['sigma']['standard deviation']
        alpha_mean, alpha_std = MDL.stats()['alpha']['mean'], MDL.stats()['alpha']['standard deviation']
        TP_min_mean, TP_min_std = MDL.stats()['TP_min']['mean'], MDL.stats()['TP_min']['standard deviation']


        x0_mean, x0_std = round(x0_mean, 2), round(x0_std, 2)
        sigma_mean, sigma_std = round(sigma_mean, 2), round(sigma_std, 2)
        alpha_mean, alpha_std = round(alpha_mean, 2), round(alpha_std, 2)
        TP_min_mean, TP_min_std = round(TP_min_mean, 2), round(TP_min_std, 2)

        # Array of best fitted parameters:
        bf_vals = [x0_mean, sigma_mean, alpha_mean, TP_min_mean]
        bf_valsUnc = [x0_std, sigma_std, alpha_std, TP_min_std]
        print("Best fit vals: ", bf_vals)
        print("Err in best fit vals: ", bf_valsUnc)
        
        
        ### store the fit parameters for plot
        x0_mean_evol.append(x0_mean)
        sigma_mean_evol.append(sigma_mean)
        alpha_mean_evol.append(alpha_mean)
        TP_min_mean_evol.append(TP_min_mean)

        x0_std_evol.append(x0_std) 
        sigma_std_evol.append(sigma_std)
        alpha_std_evol.append(alpha_std)
        TP_min_std_evol.append(TP_min_std)
        
        
        time_MCMC = (np.float(time_step)-10)*2.5*10**(-2)
        TIME.append(time_MCMC)
        ## Fitting the histograms 
        
        ## calculating the chisquare for MCMC
        ## select only positive yfit values
        xmax_mcmc,xmin_mcmc = mf.fitting_datafor_chisquare_lgpw(xfit_test,*bf_vals,norm_const)
        xfit_mcmc = np.linspace(xmin_mcmc, xmax_mcmc, len(ydata1))
        yfit_mcmc = mf.function_plot(xfit_mcmc, *popt,norm_const)
        yfit = mf.function1(xfit_mcmc , *bf_vals)

        ## the unnormalized values
        yfit_notnorm = mf.function_plot(xfit_mcmc, *bf_vals,norm_const)
        
        ## the chi square estimate: 
        chisq_MCMC = chisquare(ydata_not_normalized,yfit_notnorm)
        print("chisq MCMC=", chisquare(ydata_not_normalized,yfit_notnorm))
        
        
        plt.figure(figsize = (8,6))
        plt.subplot(111, xscale = "linear", yscale = "linear")
#         plt.rc('text',usetex=True)
        plt.step(x,ydata_not_normalized,'ro',markersize=1)
    
        plt.plot(xfit_mcmc, mf.function_plot(xfit_mcmc, *bf_vals,norm_const), "k", linewidth = 2,label='MCMC: mu=%5.2f, sigma=%5.2f, alpha=%5.2f, TP=%5.2f' % tuple(bf_vals))
    # plt.style.use('seaborn-darkgrid')
        plt.tick_params(axis='both', which='major', labelsize=17)
        plt.plot(xfit_lgpw, mf.function_plot(xfit_lgpw, *popt,norm_const),'b--',
                 label='LN+PL : mu=%5.2f, sigma=%5.2f, alpha=%5.2f, TP=%5.2f' % tuple(popt))
#         plt.plot(xfit, mf.lognormal_plot(xfit,*popt_lg,norm_const),'r-',label='LN: mu=%5.2f, sigma=%5.2f' % tuple(popt_lg))
        plt.text(-.45,3.7,'Model Selected : LN+PL',color='Blue', fontsize=12)
        plt.xlim(-.5,1.8)
        plt.ylim(-0.1,4)
        plt.legend(loc = 3)
        plt.text(.70,3.3,'alpha = '+str(alpha_mean)+'$\pm$'+str(alpha_std),color='red', fontsize=12)
        plt.text(.7,3.0,'TP = ('+str(TP_min_mean)+'$\pm$'+str(TP_min_std)+') ',color='red', fontsize=12)
        plt.text(.7,2.7,'# of Bins = '+str(b),color='red', fontsize=12)
        plt.title('Model  '+str(fileName[21]+fileName[22]+fileName[23]+'-'+fileName[25]+fileName[26]), fontsize=14)
        plt.text(-.45,3.3,r'T = %.4s'%(time_in_my)+'Myr',fontsize=15,color='black')
        plt.text(0.7,2.4,  r'$\chi^2_{MCMC}$ = '+str(round(chisq_MCMC[0],2)), color='red', fontsize=15)
        plt.text(0.7,2.1,  r'$\chi^2_{Curve-fit}$ = '+str(round(chisq_lgpw[0],2)), color='red', fontsize=15)
        
        
    # plt.axis([21.2, 23.6, 0., 5.])
        plt.xlabel("Log $\Sigma/\Sigma_{0}$", fontsize = 18)
        plt.ylabel("Log n", fontsize = 18)
        plt.tight_layout()
        outputfilename_figfile = os.path.join(save_path1,'ev_'+str(time_step)+".jpg")
        plt.savefig(outputfilename_figfile,format='jpg',dpi=300)
#         plt.show()
        plt.close()
    else: 
        print("fit lognormal_fit")
        
        
        LN_x0_mean_evol.append(popt_lg[0])
        LN_x0_std_evol.append(error_std_lg[0])

        LN_sigma_mean_evol.append(popt_lg[1])
        LN_sigma_std_evol.append(error_std_lg[1])

        time_lognomr = (np.float(time_step)-10)*2.5*10**(-2)
        TIME_LN.append(time_lognomr)
        
        

        plt.figure(figsize = (8,6))
        plt.subplot(111, xscale = "linear", yscale = "linear")
        plt.tick_params(axis='both', which='major', labelsize=17)
        #     plt.rc('text',usetex=True)
        plt.step(x,ydata_not_normalized,'ro',markersize=1)

        plt.plot(xfit_lg, mf.lognormal_plot(xfit_lg,*popt_lg,norm_const),'k--',label='LN: mu=%5.2f, sigma=%5.2f' % tuple(popt_lg))
        if popt[3]<np.max(x):
            plt.plot(xfit_lgpw, mf.function_plot(xfit_lgpw,*popt,norm_const),'b--',label='LN+PL: mu=%5.2f, sigma=%5.2f, alpha=%5.2f, TP=%5.2f' % tuple(popt))
        plt.legend(loc = 3)
#         plt.text(.65,3.0,' mu='+str(round(popt[0],2))+'and'+ ' sigma='+str(round(popt[1],2)),color='Green', fontsize=12)
        plt.text(-.45,3.7,'Model Selected : LN',color='Blue', fontsize=12)
        plt.text(.7,2.7,'# of Bins = '+str(b),color='red', fontsize=12)
        plt.title('Model  '+str(fileName[21]+fileName[22]+fileName[23]+'-'+fileName[25]+fileName[26]), fontsize=14)
        plt.text(-.45,3.3,r'T = %.4s'%(time_in_my)+'Myr',fontsize=15,color='black')
        plt.text(0.7,2.4,  r'$\chi^2_{lognormal}$ = '+str(round(chisq_lg[0],2)), color='red', fontsize=15)
        plt.text(0.7,2.1,  r'$\chi^2_{lognormal+powerlaw}$ = '+str(round(chisq_lgpw[0],2)), color='red', fontsize=15)
        plt.xlim(-.5,1.8)
        plt.ylim(-0.1,4)
        
        # plt.axis([21.2, 23.6, 0., 5.])
        plt.xlabel("Log $\Sigma/\Sigma_{0}$", fontsize = 18)
        plt.ylabel("Log n", fontsize = 18)
        plt.tight_layout() 
        outputfilename_figfile = os.path.join(save_path1,'ev_'+str(time_step)+".jpg")
        plt.savefig(outputfilename_figfile,format='jpg',dpi=300)
#         plt.show()
        plt.close()
    
    


In [7]:
number_of_timestep = len(list)
print(number_of_timestep)

# number_of_orbit = 10
start = time.time()
print("Plotting now...")
if __name__ == '__main__':
    pool = mp.Pool()
    pool.map(do_plot, range(0,number_of_timestep,1))
print("End of simulation")    
end = t.time()
T = end-start
print("Total time for parallel run %.2f"%T, 'sec')


324
Plotting now...
../evolution_data/TP_B16_v20/colden_ev052.dat
../evolution_data/TP_B16_v20/colden_ev031.dat
../evolution_data/TP_B16_v20/colden_ev010.dat
../evolution_data/TP_B16_v20/colden_ev073.dat
31
10
73
52
[ 0.5796557   0.33833243 -4.5         0.6       ]
<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power_divergenceResult(statistic=4.873084660817792, pvalue=1.0)
chisq lognormal(LC method)= Power_divergenceResult(statistic=4.8730824142372073, pvalue=1.0)
fit lognormal_fit


/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


[ 0.61628909  0.21562415 -4.5         0.6       ]
<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power_divergenceResult(statistic=3.9906593736988811, pvalue=1.0)
chisq lognormal(LC method)= Power_divergenceResult(statistic=3.9906474601372777, pvalue=1.0)
fit lognormal_fit
[  6.57473110e-01   3.11389349e-03  -4.50000000e+00   6.00000000e-01]
<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power_divergenceResult(statistic=5.2232604037726033e-17, pvalue=nan)
chisq lognormal(LC method)= Power_divergenceResult(statistic=7.326909878779861e-17, pvalue=nan)
fit lognormal_fit
[ 0.62951496  0.14070915 -4.5         0.6       ]
<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power_divergenceResult(statistic=6.402442034727148, pvalue=1.0)
chisq lognormal(LC method)= Power_divergenceResult(statistic=6.4024409980553614, pvalue=1.0)
fit lognormal_fit
../evolution_data/TP_B16_v20/colden_ev032.dat
../evolution_data/TP_B16_v20/colden_ev074.dat
../evolution_data/

chisq lognormal(LC method)= Power_divergenceResult(statistic=4.9807137408652151, pvalue=1.0)
fit lognormal_fit
[ 0.62882593  0.15507976 -4.5         0.6       ]
<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power_divergenceResult(statistic=1.5771674349939979, pvalue=1.0)
chisq lognormal(LC method)= Power_divergenceResult(statistic=1.5771695024332586, pvalue=1.0)
fit lognormal_fit
16
[ 0.60982647  0.18297027 -4.5         0.6       ]
<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power_divergenceResult(statistic=5.1243051784871287, pvalue=1.0)
chisq lognormal(LC method)= Power_divergenceResult(statistic=5.1243107329483495, pvalue=1.0)
fit lognormal_fit
../evolution_data/TP_B16_v20/colden_ev038.dat
38
../evolution_data/TP_B16_v20/colden_ev080.dat
../evolution_data/TP_B16_v20/colden_ev059.dat
../evolution_data/TP_B16_v20/colden_ev017.dat
80
59
[ 0.5584209   0.28689096 -4.5         0.6       ]
<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power

64
[ 0.61091089  0.17593821 -4.5         0.6       ]
<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power_divergenceResult(statistic=4.1080145424703396, pvalue=1.0)
chisq lognormal(LC method)= Power_divergenceResult(statistic=4.1080146878843076, pvalue=1.0)
fit lognormal_fit
[ 0.63827781  0.17954726 -4.5         0.6       ]
<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power_divergenceResult(statistic=3.6169659610712026, pvalue=1.0)
chisq lognormal(LC method)= Power_divergenceResult(statistic=3.6169721342061547, pvalue=1.0)
fit lognormal_fit
../evolution_data/TP_B16_v20/colden_ev044.dat
../evolution_data/TP_B16_v20/colden_ev023.dat
44
../evolution_data/TP_B16_v20/colden_ev086.dat
../evolution_data/TP_B16_v20/colden_ev065.dat
[ 0.57107498  0.20976594 -4.5         0.6       ]
<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power_divergenceResult(statistic=8.1297006927112605, pvalue=1.0)
chisq lognormal(LC method)= Power_divergenceResult(statis

...

chisq lognormal(LC method)= Power_divergenceResult(statistic=1.2914180274379048, pvalue=1.0)
fit lognormal + Powerlaw
Initial params [0.61468050898221471, 0.28902875562303065, -4.78507612312481, 0.52025906636407471]
 [-----------------93%---------------   ] 18799 of 20000 complete in 22.5 secBest fit vals:  [0.58999999999999997, 0.31, -3.2799999999999998, 0.58999999999999997]
Err in best fit vals:  [0.01, 0.0, 0.059999999999999998, 0.01]
chisq MCMC= Power_divergenceResult(statistic=54.556723313478471, pvalue=1.0)
 [-----------------95%----------------  ] 19141 of 20000 complete in 23.0 sec../evolution_data/TP_B16_v20/colden_ev331.dat
331
[ 0.58938964  0.31105783 -3.19378165  0.59957935]
<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power_divergenceResult(statistic=17.445536269912274, pvalue=1.0)
chisq lognormal(LC method)= Power_divergenceResult(statistic=39.575757697954543, pvalue=1.0)
fit lognormal + Powerlaw
Initial params [0.58938963723352189, 0.3110578265366743, -3

 [-----------------63%----              ] 12704 of 20000 complete in 16.0 secBest fit vals:  [0.58999999999999997, 0.31, -2.8900000000000001, 0.62]
Err in best fit vals:  [0.01, 0.0, 0.080000000000000002, 0.01]
chisq MCMC= Power_divergenceResult(statistic=18.895514455056013, pvalue=1.0)
 [-----------------82%-----------       ] 16490 of 20000 complete in 20.0 sec../evolution_data/TP_B16_v20/colden_ev333.dat
 [-----------------65%----              ] 13094 of 20000 complete in 16.5 sec333
[ 0.58906851  0.31207714 -2.85142615  0.61432415]
<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power_divergenceResult(statistic=8.4134281580986539, pvalue=1.0)
chisq lognormal(LC method)= Power_divergenceResult(statistic=19.976186608961022, pvalue=1.0)
fit lognormal + Powerlaw
Initial params [0.58906851224578072, 0.31207714379865098, -2.8514261528184708, 0.6143241537914742]
 [--------         22%                  ] 4523 of 20000 complete in 4.0 sececBest fit vals:  [0.59999999999999998,

 [---------------  40%                  ] 8098 of 20000 complete in 10.0 secBest fit vals:  [0.60999999999999999, 0.28999999999999998, -5.0199999999999996, 0.42999999999999999]
Err in best fit vals:  [0.0, 0.0, 0.080000000000000002, 0.029999999999999999]
chisq MCMC= Power_divergenceResult(statistic=0.89188753067212923, pvalue=1.0)
 [---------------- 42%                  ] 8476 of 20000 complete in 10.5 sec../evolution_data/TP_B16_v20/colden_ev281.dat
281
[ 0.60917889  0.29001268 -4.92552185  0.5       ]
<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power_divergenceResult(statistic=0.88256210621244602, pvalue=1.0)
chisq lognormal(LC method)= Power_divergenceResult(statistic=1.905784826008549, pvalue=1.0)
fit lognormal + Powerlaw
Initial params [0.60917889229375899, 0.29001267738892189, -4.9255218536783438, 0.50000000000000011]
 [-----------------91%--------------    ] 18373 of 20000 complete in 19.5 secBest fit vals:  [0.59999999999999998, 0.28999999999999998, -3.9199999

<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power_divergenceResult(statistic=3.3996113903540146, pvalue=1.0)
chisq lognormal(LC method)= Power_divergenceResult(statistic=22.087024267991225, pvalue=1.0)
fit lognormal + Powerlaw
Initial params [0.59594464930716962, 0.30456367481925456, -4.0333421269518386, 0.56946515901754802]
 [-----------------100%-----------------] 20000 of 20000 complete in 21.0 secBest fit vals:  [0.59999999999999998, 0.28999999999999998, -3.27, 0.55000000000000004]
Err in best fit vals:  [0.0, 0.0, 0.17999999999999999, 0.01]
chisq MCMC= Power_divergenceResult(statistic=0.95893726541609192, pvalue=1.0)
../evolution_data/TP_B16_v20/colden_ev298.dat
298
[ 0.59976664  0.29230172 -3.26357288  0.55367512]
<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power_divergenceResult(statistic=0.91217894663378862, pvalue=1.0)
chisq lognormal(LC method)= Power_divergenceResult(statistic=5.2720785210967982, pvalue=1.0)
fit lognormal + Powerlaw
Initia

fit lognormal + Powerlaw
Initial params [0.59394212444854322, 0.30738344818284868, -3.9214532249284288, 0.57369550898471344]
 [-----------------97%----------------- ] 19513 of 20000 complete in 23.0 secBest fit vals:  [0.59999999999999998, 0.28999999999999998, -3.6200000000000001, 0.55000000000000004]
Err in best fit vals:  [0.0, 0.0, 0.16, 0.01]
chisq MCMC= Power_divergenceResult(statistic=1.3709939417971979, pvalue=1.0)
 [-----------------99%----------------- ] 19909 of 20000 complete in 23.5 sec../evolution_data/TP_B16_v20/colden_ev303.dat
 [-----------------100%-----------------] 20000 of 20000 complete in 23.6 sec303
[ 0.5962047   0.2937432  -3.70328123  0.55025705]
<class 'numpy.ndarray'>
chisq lognormal+powerlaw (LC method)= Power_divergenceResult(statistic=1.5158228118684511, pvalue=1.0)
chisq lognormal(LC method)= Power_divergenceResult(statistic=6.3385203923689772, pvalue=1.0)
fit lognormal + Powerlaw
Initial params [0.59620470136347159, 0.29374319845267904, -3.70328123146017

In [8]:
import cv2
print("Making the Video")

image_folder = save_path1 
video_name = 'video_'+ Model1 +'.mov'
if os.path.isfile(video_name):
    print("Removing the old video and making the new one")
    os.remove(video_name)

image = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
## sorting the images in ascending order of time
images= sorted(image, key=lambda x: int(x.replace("ev_", '').replace('.jpg', '')))
# images =sorted(image)
# print(images)
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, -1, 13, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

print("The video is done")

Making the Video


NameError: name 'Model1' is not defined

In [ ]:
## This is delete all the images to save space
fileList = os.listdir(save_path1)

for fileName in fileList:
#     print(fileName)
    os.remove(save_path1+"/"+fileName)

In [ ]:
print(np.log(10))